In [2]:
import pandas as pd

# Cargar los archivos CSV
canciones_df = pd.read_csv('canciones22.csv')
perfiles_df = pd.read_csv('cluster2.csv')

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [15]:
import pandas as pd
import numpy as np

def assign_song_contextual(user, songs, assigned_songs, popularity_weight=4):
    """
    Asigna una canción a un usuario basado en:
    - Género musical favorito (`SuperGenre`).
    - Estado de ánimo (`music_Influencial_mood`).
    - Frecuencia de escucha (`music_lis_frequency`).
    - Popularidad, características avanzadas y penalización por repetición.
    - Artistas relevantes según la edad del usuario.
    """
    # Artistas relevantes según la edad (extensivo y realista)
    age_specific_artists = {
        '60+': [
        'Frank Sinatra', 'Elvis Presley', 'The Beatles', 'Nat King Cole', 'Ray Charles', 'Ella Fitzgerald',
        'Louis Armstrong', 'Aretha Franklin', 'Johnny Cash', 'Dean Martin', 'Bob Dylan', 'Patsy Cline',
        'The Platters', 'Roy Orbison', 'Glenn Miller', 'Billie Holiday', 'Bing Crosby', 'The Everly Brothers',
        'Tony Bennett', 'Fats Domino', 'Sam Cooke', 'Duke Ellington', 'Brenda Lee', 'Ricky Nelson', 'Joan Baez',
        'Perry Como', 'Connie Francis', 'Andy Williams', 'Peter, Paul and Mary', 'Dusty Springfield',
        'Tammy Wynette', 'Loretta Lynn', 'Merle Haggard', 'Willie Nelson', 'Charlie Parker', 'Benny Goodman',
        'Otis Redding', 'The Supremes', 'Marvin Gaye', 'Bobby Darin', 'The Mamas & The Papas', 'The Beach Boys',
        'Carpenters', 'Simon & Garfunkel', 'Jim Croce', 'Carole King', 'Etta James', 'Stevie Wonder',
        'Paul Anka', 'Neil Diamond', 'Johnny Mathis', 'Janis Joplin', 'Led Zeppelin', 'The Rolling Stones',
        'Pink Floyd', 'Eric Clapton', 'Crosby, Stills, Nash & Young', 'Cat Stevens', 'Rod Stewart', 'Joni Mitchell',
        'The Temptations', 'Jackson 5', 'Al Green', 'Van Morrison', 'The Doors', 'The Who', 'Cream', 
        'Chicago', 'Blood, Sweat & Tears', 'George Harrison', 'Ringo Starr', 'Jefferson Airplane'
    ],
    '35~60': [
        'Madonna', 'Michael Jackson', 'Prince', 'U2', 'Queen', 'Phil Collins', 'Whitney Houston',
        'Bon Jovi', 'Bruce Springsteen', 'George Michael', 'Tina Turner', 'Billy Joel', 'Fleetwood Mac',
        'ABBA', 'The Police', 'Aerosmith', 'Journey', 'Cyndi Lauper', 'The Cure', 'Depeche Mode',
        'The Smiths', 'Simple Minds', 'Duran Duran', 'Tears for Fears', 'Roxette', 'Janet Jackson',
        'Genesis', 'Eurythmics', 'Dire Straits', 'Heart', 'Def Leppard', 'Van Halen', 'Scorpions', 'Europe',
        'Sade', 'Richard Marx', 'Bryan Adams', 'Sting', 'A-ha', 'Wham!', 'Foreigner', 'The Bangles', 'INXS',
        'Guns N\' Roses', 'Metallica', 'AC/DC', 'REM', 'Pat Benatar', 'Talking Heads', 'Blondie',
        'Pet Shop Boys', 'Culture Club', 'The Human League', 'Level 42', 'Rick Astley', 'Erasure',
        'Berlin', 'Billy Idol', 'REO Speedwagon', 'Boston', 'Kenny Loggins', 'Toto', 'Lionel Richie',
        'Paul McCartney', 'Stevie Nicks', 'Joe Cocker', 'Chaka Khan', 'Boy George', 'The Cranberries',
        'Shania Twain', 'Natalie Cole', 'Kool & The Gang', 'Earth, Wind & Fire', 'Gloria Estefan',
        'Barry White', 'Donna Summer', 'The Alan Parsons Project'
    ],
    '20~35': [
        'Drake', 'Ariana Grande', 'The Weeknd', 'Ed Sheeran', 'Taylor Swift', 'Dua Lipa', 'Post Malone',
        'Doja Cat', 'Bad Bunny', 'Rosalía', 'Billie Eilish', 'Harry Styles', 'BTS', 'BLACKPINK',
        'Imagine Dragons', 'Coldplay', 'Justin Bieber', 'Selena Gomez', 'Shawn Mendes', 'Camila Cabello',
        'Khalid', 'Maroon 5', 'Halsey', 'Lizzo', 'J Balvin', 'Karol G', 'Anitta', 'Sam Smith', 'Olivia Rodrigo',
        'Lil Nas X', 'SZA', 'Tate McRae', 'The Kid LAROI', 'Conan Gray', 'Charlie Puth', 'Alan Walker',
        'Kygo', 'Zedd', 'Marshmello', 'Jonas Brothers', 'AJR', 'Twenty One Pilots', '5 Seconds of Summer',
        'Tame Impala', 'Lauv', 'Troye Sivan', 'Glass Animals', 'Rina Sawayama', 'Anne-Marie', 'Saweetie',
        'Becky G', 'Sebastián Yatra', 'Farruko', 'Ozuna', 'Natti Natasha', 'Rauw Alejandro', 'Megan Thee Stallion',
        'Cardi B', 'Nicki Minaj', 'Machine Gun Kelly', 'Hozier', 'Of Monsters and Men', 'Bastille', 
        'Vance Joy', 'The Chainsmokers', 'Zara Larsson', 'KAROL G', 'Maneskin'
    ],
    '12~20': [
        'Billie Eilish', 'Olivia Rodrigo', 'BTS', 'BLACKPINK', 'Harry Styles', 'Conan Gray', 'Tate McRae',
        'The Kid LAROI', 'Ava Max', 'Jxdn', 'TXT', 'Stray Kids', 'NCT', 'ITZY', 'Aespa', 'Seventeen', 'TREASURE',
        'NewJeans', 'The Weeknd', 'Doja Cat', 'Lil Nas X', 'SZA', 'Rosalía', 'Bad Bunny', 'Karol G', 'J Balvin',
        'Rauw Alejandro', 'Shawn Mendes', 'Camila Cabello', 'Halsey', 'Khalid', 'TOMORROW X TOGETHER',
        'TREASURE', 'TWICE', 'Le Sserafim', 'Ive', 'Kehlani', 'Sabrina Carpenter', 'Joshua Bassett',
        'Madison Beer', 'Alec Benjamin', 'Lana Del Rey', 'Arctic Monkeys', 'Imagine Dragons', 
        'Paramore', 'The 1975', 'Coldplay', 'Tame Impala', 'Twenty One Pilots', 'Alan Walker',
        'Kygo', 'Zedd', 'Marshmello', 'AJR', 'Becky G', 'Anne-Marie', 'Zara Larsson', 'Ellie Goulding'
    ]
    }
# Inicializar puntuación
    songs['score'] = 0

    # Paso 1: Filtro por género musical favorito
    if pd.notna(user['fav_music_genre']) and user['fav_music_genre'].lower() != 'all':
        songs['score'] += songs['SuperGenre'].str.fullmatch(user['fav_music_genre'], case=False, na=False) * 20

    # Paso 2: Priorización por popularidad
    songs['score'] += (songs['popularity'] / 10) * popularity_weight

    # Paso 3: Priorización por artistas relevantes
    if pd.notna(user['Age']) and user['Age'] in age_specific_artists:
        relevant_artists = age_specific_artists[user['Age']]
        songs['score'] += songs['artists'].str.contains('|'.join(relevant_artists), case=False, na=False) * 25

    # Paso 4: Priorización por estado de ánimo
    if pd.notna(user['music_Influencial_mood']):
        moods = [mood.strip().lower() for mood in user['music_Influencial_mood'].split(",")]
        mood_priority = {
            'motivational': {'energy': (0.7, 1.0), 'valence': (0.5, 1.0), 'weight': 20},
            'relaxation': {'acousticness': (0.6, 1.0), 'energy': (0.0, 0.4), 'weight': 15},
            'sadness': {'valence': (0.0, 0.3), 'energy': (0.0, 0.5), 'weight': 10},
            'social': {'danceability': (0.7, 1.0), 'energy': (0.5, 1.0), 'weight': 10}
        }
        for mood in moods:
            if mood in mood_priority:
                conditions = mood_priority[mood]
                for feature, values in conditions.items():
                    if feature != 'weight':
                        min_val, max_val = values
                        songs['score'] += ((songs[feature] >= min_val) & (songs[feature] <= max_val)) * conditions['weight']

    # Paso 5: Priorización por frecuencia de escucha
    if pd.notna(user['music_lis_frequency']):
        frequencies = [freq.strip().lower() for freq in user['music_lis_frequency'].split(",")]
        frequency_weights = {
            'workout session': {'energy': (0.8, 1.0), 'danceability': (0.6, 1.0), 'weight': 15},
            'social gatherings': {'danceability': (0.7, 1.0), 'weight': 12},
            'while traveling': {'tempo': (100, 200), 'weight': 10},
            'office hours': {'instrumentalness': (0.3, 1.0), 'valence': (0.4, 1.0), 'weight': 8},
            'study hours': {'acousticness': (0.4, 1.0), 'weight': 8},
            'leisure time': {'valence': (0.5, 1.0), 'energy': (0.3, 0.6), 'weight': 6}
        }
        for freq in frequencies:
            if freq in frequency_weights:
                conditions = frequency_weights[freq]
                for feature, values in conditions.items():
                    if feature != 'weight':
                        min_val, max_val = values
                        songs['score'] += ((songs[feature] >= min_val) & (songs[feature] <= max_val)) * conditions['weight']

    # Paso 6: Penalización por repetición de artistas y canciones
    songs.loc[songs['track_id'].isin(assigned_songs), 'score'] -= 100

    # Paso 7: Selección final basada en puntuación
    best_song = songs.loc[songs['score'].idxmax()]
    if best_song['score'] > 0:
        return pd.Series([best_song['track_name'], best_song['track_id']])

    # Fallback si no hay coincidencias
    fallback_songs = songs[~songs['track_id'].isin(assigned_songs)]
    if not fallback_songs.empty:
        fallback_song = fallback_songs.sort_values(by='popularity', ascending=False).iloc[0]
        return pd.Series([fallback_song['track_name'], fallback_song['track_id']])

    return pd.Series([None, None])

# Procesar usuarios y asignar canciones
assigned_songs = set()
for index, user in perfiles_df.iterrows():
    result = assign_song_contextual(user, canciones_df, assigned_songs)
    perfiles_df.at[index, 'Assigned_Song'] = result[0]
    perfiles_df.at[index, 'Track_ID'] = result[1]
    if result[1]:
        assigned_songs.add(result[1])

# Guardar resultados
output_file = 'perfiles_con_canciones.csv'
perfiles_df.to_csv(output_file, index=False)
print(f"Archivo guardado exitosamente en: {output_file}")

Archivo guardado exitosamente en: perfiles_con_canciones.csv


## Asignación cancion 

In [48]:
perfiles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9469 entries, 0 to 9468
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     9469 non-null   object
 1   Gender                  9469 non-null   object
 2   fav_music_genre         9469 non-null   object
 3   music_time_slot         9469 non-null   object
 4   music_Influencial_mood  9469 non-null   object
 5   music_lis_frequency     9469 non-null   object
 6   Assigned_Song           9469 non-null   object
 7   Track_ID                9469 non-null   object
dtypes: object(8)
memory usage: 591.9+ KB


In [52]:
perfiles_df['fav_music_genre'].value_counts()

fav_music_genre
Música Cultural y Melódica         2415
Ritmos Globales y Festivos         1836
Pop Urbano y Ritmos Modernos       1773
Ambiental y Relajante              1747
All                                 452
Rock Electrónico y Energético       439
Electrónica y House Alternativo     429
Acústica y Nostálgica               378
Name: count, dtype: int64